In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.filterwarnings('ignore')

In [8]:
from sklearn.linear_model import Ridge

data = pd.read_csv('drop_outlier_scaled.csv')
X = data.drop(['quality'], axis=1)
y = data['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

ridge = Ridge(alpha=0.1)

ridge.fit(X_train, y_train)

pre = ridge.predict(X_test) 
print(f'테스트 세트의 정확도 : {ridge.score(X_test, y_test):.4f}')
print(f'pre :\n{pre[:10]}')
print(f'y   :\n{y_test.values[:10]}')

테스트 세트의 정확도 : 0.3636
pre :
[5.7531278  5.71230526 5.11020115 5.13866842 6.38729569 5.4422123
 5.41275402 5.22601172 5.64215536 5.73879546]
y   :
[6 6 5 5 6 6 6 5 5 6]
